## Домашнее задание 3 (10 баллов)

## НИУ ВШЭ, 2023-24 учебный год

## О задании

В этом домашнем задании вы получите опыт подготовки данных и EDA (Exploratory Data Analysis), приблизившись к реальной работе с данными с помощью уже знакомых вам инструментов: numpy, pandas, matplotlib и seaborn, а также познакомитесь с plotly.

EDA - важный этап работы с данными. Именно в процессе EDA вы можете выяснить особенности датасета, найти зависимости в данных, протестировать гипотезы о взаимосвязях признаков и целевой переменной. В реальности вы сами будете определять, что и как проверять в рамках EDA, однако в данной работе гипотезы будут выдвинуты за вас автором задания. Также и типы графиков, которые вы будете использовать уже предопределены за вас. Выполняя задания по визуализациям, обращайте внимание на референсные картинки, которые были оставлены автором. Они показывают, какой примерно результат от вас ожидается.

Это задание будет включать в себя построение некоторого количества графиков, потому вот основные правила, в соответствии с которыми графики должны быть оформлены:
1. Любой график всегда имеет название.
2. Оси у графика подписаны (исключение - график делит название оси с соседним).
3. Подписи делаются на одном языке (либо английский, либо русский), но не на их смеси.
4. Если вы выводите несколько графиков, то они размещаются в рамках одного объекта matplotlib.pyplot.figure. Вы не выводите графики последовательно, а смотрите, как их разместить сразу все за раз.
5. Отдельные графики (если графиков на картинке больше 1) также имеют свои названия.
6. Если на графике есть несколько различных объектов, то для их различения используется легенда. Помещайте легенду в место, где она в наименьшей мере мешает восприятию самого графика.
7. Если есть легенда, то она должна иметь навание, а также ее содержание должно быть адекватными (пожалейте проверяющих, они не хотят разбирать, а к чему относится зеленая линия, которая на легенде помечена меткой "0").
8. Используйте цветовые гаммы, чтобы объекты на графиках отличались цветом.
9. Текст на визуализации должен быть читаем. Если вы вывели названия категорий в качестве подписей на оси Х, а эти названия наезжают друг на друга, то вам нужно что-то с этим сделать. Нечитаемый или плохо читаемый (очень маленький, невероятно огромный) текст ведет к снижению баллов.

*Тут правило про плагиат, которое вам итак известно - обнуляются все работы, которые подозреваются в некорректных заимствованиях!* Помните, что просто скопировать код из интернета и указать источник - это не ваша самостоятельная работа, а автора кода. Измените хоть что-нибудь в коде.

## О данных

Будем работать с [датасетом физической активности](https://www.kaggle.com/datasets/uciml/human-activity-recognition-with-smartphones).

В этом датасете собраны данные 30 человек, выполняющих различные действия со смартфоном на поясе. Данные записывались с помощью датчиков (акселерометра и гироскопа) в этом смартфоне. Были зафиксированы: "3-осевое линейное ускорение" (_tAcc-XYZ_) и "3-осевая угловая скорость" (_tGyro-XYZ_).

1. Все сигналы отсортированы по времени, пропущены через фильтр шума.
2. Все наблюдения были разделены на окна по 128 сигналов в каждом, причем окна пересекаются на 50% (грубо говоря, с 1 по 128 сигнал в 1 окно, с 64 до 192 сигнала - во 2 окно и так далее).
3. Линейное ускорение было разделено на две составляющие (с помощью низкочастотного фильтра): ускорение самого тела, а также гравитационная составляющая. Будем называть их ускорением тела и гравитационным ускорением.
4. К получившимся окнам применяли различные функции для получения следующих показателей:
    - max
    - min
    - mad (median)
    - mean
    - std
    - skewness (коэффициент асимметричности распределения признака в окне)
    - etc. (подробнее, на сколько это возможно, указано на [страничке датасета](https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones))
5. Величина (magnitude) сигнала также определялась через евклидову норму всех значений из вектора для одного окна.
6. К окнам было применено преобразование Фурье, чтобы получить частоты. Из частот были извелченыте же показатели, плюс новые:
    - bandsEnergy (энергия частотного интервала)
7. Были посчитаны углы между векторами соответствующих окон.
8. Для ускорений были посчитаны производные - рывки, к которым применялись те же функции.

Итог: в датасете каждый признак - это какая-то операция над вектором из 128 чисел, которые соответствуют замерам определенного человека занятого какой-то активностью на протяжении 2.56 секунд.

Как читать названия признаков? Примеры:
- `tBodyAccMag-mean()` - это вектор ускорений тела (без гравитационной составляющей этого ускорения), из которого взяли евклидову норму, а затем усреднили, чтобы получить скаляр
- `fBodyAcc-bandsEnergy()-1,8.2` - это вектор ускорения тела, к которому применили преобразование Фурье, преобразовав его в вектор частот, а затем посчитали на этом векторе энергию данного частотного интервала в определенном диапазоне частот (в данном случае 1 - 8.2)
- `angle(X,gravityMean)` - это угол между осью Х и вектором усредненных значений гравитационного ускорения из соответствующего окна

Вероятно, сначала покажется сложным, но мы разберемся с некоторыми признаками в отдельности и вы сможете понять их смысл, уже исходя из фактических соотношений.

## Предобработка данных

In [ ]:
# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Загрузил архив на гитхаб, чтобы оттуда быстро импортировать файлы
!wget https://github.com/PiginIvan/Archives/raw/main/archive.zip
!unzip /content/archive.zip

In [ ]:
# Читаем данные из таблиц
train = pd.read_csv('train.csv')
test = pd.read_csv("test.csv")

In [ ]:
# Размер данных train
train.shape

В датасете 563 признака, включая целевую переменную. Столько признаков нам не нужно. Мы за вас сделали подвыборку признаков для удобства работы с ними.

In [ ]:
cols_subset = [
    'tBodyAccMag-mean()',  # усредненная величина ускорения тела
    'tBodyGyroJerk-mad()-X',  # медианная величина рывка тела по оси Х
    'tGravityAcc-min()-X',  # минимум гравитационной составляющей ускорения по оси Х
    'tBodyAcc-max()-X',  # максимальная величина ускорения тела по оси Х
    'fBodyAcc-bandsEnergy()-1,8.2',  # энергия ускорения тела в интервале частоты
    'angle(X,gravityMean)',  # угол между осью Х и усредненной гравитационной составляющей ускорения
    'angle(Y,gravityMean)',  # угол между осью Y и усредненной гравитационной составляющей ускорения
    'angle(Z,gravityMean)',  # угол между осью Z и усредненной гравитационной составляющей ускорения,
    'fBodyAcc-skewness()-X',  # асимметричность частоты ускорения тела по оси Х
    'subject',  # номер испытуемого
    'Activity',  # название вида деятельности (целевая переменная)
]

In [ ]:
# Предвыборка необходимых признаков
train = train[cols_subset]
test = test[cols_subset]

### Задание 1 (0.5 балла)

Посмотрите на пропуски и дубли в данных train и test. Есть ли они? Если есть, напишите, с чем это может быть связано? Если нет, то просто напишите, что пропусков и/или дублей нет.

In [ ]:
# Проверяем датасеты на наличие дубликатов и пропусков
print('Количество дубликатов в train датасете:', train.duplicated().sum())
print('Есть ли в train датасете пропуски?', np.any(train.isna().values))
print('Количество дубликатов в test датасете:', test.duplicated().sum())
print('Есть ли в test датасете пропуски?', np.any(test.isna().values))

# Дубликатов и пропусков в датасетах нет

### Задание 2 (0.5 балла)

Выведите основные статистики данных: среднее, медиана, перцентили (0.25, 0.5, 0.75), минимум, максимум, стандартное отклонение. Посмотрите на таблицу и напишите, видите ли вы в ней что-нибудь интересное.

In [ ]:
# Значения данных таблицы train
train_statistics = train.describe()[1:]
train_median = train.median(numeric_only=True)
train_statistics = pd.concat([train_statistics, train_median.to_frame(name='median').T])
train_statistics

# Все значения находятся в диапазоне от -1 до 1, можно сделать выводы, что данные масштабированы

In [ ]:
# Значения данных таблицы test
test_statistics = test.describe()[1:]
test_median = test.median(numeric_only=True)
test_statistics = pd.concat([test_statistics, test_median.to_frame(name='median').T])
test_statistics

# Все значения находятся в диапазоне от -1 до 1, можно сделать выводы, что данные масштабированы

### Задание 3 (0.5 балла)

Данные сбалансированы? У вас есть 6 категорий активности (целевая переменная). Если одна из категорий представлена большим количеством наблюдений, а остальные - нет, то это обстоятельство, о котором хотелось бы знать заранее. Гиперболизированный пример: представьте, что у вас 1000 наблюдений, разделенных на два класса, и только 5 принадлежат ко второму классу. Как учить модель на такой выборке? Есть сильное подозрение, что многие модели будут склонны вырождаться в константу. Отсюда и необходимость проверять баланс классов.

Выведите количество наблюдений для всех категорий активности в train и test. Посчитайте разницу в процентах между самой крупной категорией и самой маленькой (самая большая категория - это 100%). Если разница (округленная до ближайшего целого числа) не превышает 30%, то давайте для наших целей считать, что данные сбалансированы. Напишите, сбалансированы ли данные в train и test.

In [ ]:
# Находим разницу между самой крупной и самой маленькой категориями в train
train_activity = train.Activity.value_counts()
min_train_category = train_activity.min()
max_train_category = train_activity.max()
train_ans = int((max_train_category - min_train_category) / max_train_category * 100 + 0.5)
print(f'Разница между самой крупной и самой маленькой категориями в train: {train_ans} %')

# Итоговое значение в train '30 %', значит данные сбалансированы

In [ ]:
# Находим разницу между самой крупной и самой маленькой категориями в test
test_activity = test.Activity.value_counts()
min_test_category = test_activity.min()
max_test_category = test_activity.max()
test_ans = int((max_test_category - min_test_category) / max_test_category * 100 + 0.5)
print(f'Разница между самой крупной и самой маленькой категориями в test: {test_ans} %')

# Итоговое значение в test '22 %', значит данные сбалансированы

### Задание 4 (0.5 балла)

Посчитайте корреляции всех признаков (включая таргет) со всеми другими. Какими получились значения? Есть идеи, почему так? Напишите ваши мысли.

In [ ]:
# Кодируем target числами, для подсчета корреляции и считаем ее методом corr
train_temp = train.copy()
train_temp['target_in_nums'] = train_temp['Activity'].map(dict(zip(sorted(train_activity.index), range(6))))
train_temp.corr(numeric_only=True)

# Есть признаки, которые хорошо коррелируют между собой, в то время как, есть и признаки, которые плохо коррелируют между собой.
# Это может быть связано с тем, что некоторые признаки имеют схожий смысл, а также, что определенные активности связаны с определенными признаками

In [ ]:
# Кодируем target числами, для подсчета корреляции и считаем ее методом corr
test_temp = test.copy()
test_temp['target_in_nums'] = test_temp['Activity'].map(dict(zip(sorted(test_activity.index), range(6))))
test_temp.corr(numeric_only=True)

# Есть признаки, которые хорошо коррелируют между собой, в то время как, есть и признаки, которые плохо коррелируют между собой.
# Это может быть связано с тем, что некоторые признаки имеют схожий смысл, а также, что определенные активности связаны с определенными признаками

Поздравляем! Вы предобработали данные для максимально комфортного их последующего анализа. Теперь вы готовы приступить к Exploratory Data Analysis!

## Exploratory Data Analysis

### Задание 5 (0.5 балла)

Вы проверили основную информацию о данных и составили свое первое впечатление о них. Теперь визуализации.

Начнем рисовать графики с небольшой разминки. Выберите две любые математические функции (отличные от линейной) и нарисуйте их на одном графике. Подпишите каждый из графиков, чтобы было понятно, какие функции вы выбрали. Ниже будет пример.

In [ ]:
# Строим графики для функций y=cos(x) и y=-ln(x) с помощью lineplot
PI = np.pi
x = np.arange(-2 * PI, 2 * PI, 0.01)
y_cos = pd.Series(np.cos(x))
y_minus_ln = pd.Series(-np.log(x))
sns.lineplot(x=x, y=y_cos, label='y=cos(x)')
sns.lineplot(x=x, y=y_minus_ln, label='y=-ln(x)')
x_labels = ['$-2\pi$', '$-3\pi/2$', '$-\pi$', '$-\pi/2$', '$0$', '$\pi/2$', '$\pi$', '$3\pi/2$', '$2\pi$']
plt.xlabel('x')
plt.ylabel('y')
plt.xticks(ticks=np.linspace(-2 * np.pi, 2 * np.pi, 9), labels=x_labels)
plt.axvline(x=0)
plt.axhline(y=0)
plt.grid(visible=True)
plt.title('Math functions')
plt.legend(loc='upper left')
plt.show()


### Задание 6 (0.5 балла)

Ранее мы уже проверили сбалансированность данных по разным категориям активностей в train и test сетах. Теперь визуализируйте количество сэмплов в каждой из категорий отдельно для train и test сетов, как в примере ниже. Проследите, чтобы порядок категорий слева и справа был одинаков. Иначе не очень удобно их сравнивать (когда они в разных местах двух графиков). Также заметьте, что в примере ось Х в одинаковом масштабе, что тоже удобно для сравнения.

Вам может пригодиться:
- plt.barh или ax.barh

In [ ]:
# Строим график с помощью barh для отображения количества значений различных активностей на обоих датасетах
fig, (first, second) = plt.subplots(1, 2, figsize=(15, 5))
fig.suptitle('Target variables distribution')
first.barh(train_activity.index, width=train_activity[train_activity.index], color='blue')
first.set_title('Train set')
first.invert_xaxis()
second.barh(train_activity.index, width=test_activity[train_activity.index], color='red')
second.set_title('Test set')
second.yaxis.tick_right()
second.set_xlim(first.get_xlim()[::-1])
fig.tight_layout()
plt.show()

Правда на таких графиках проще проверять баланс классов таргета?)

In [ ]:
# Да, данные графики более точно отображают баланс классов таргета

### Задание 7 (0.5 балла)

Ранее вы выводили таблицу корреляций. Однако просто таблица корреляций не очень удобна для анализа. Что, если признаков было бы больше? Визуализируйте heatmap корреляций. Подпишите значения корреляций, округленные до 2 знаков после запятой, на графике и оставьте только те значения, которые по модулю не меньше 0.5. Будем считать такие корреляции существенными. Если будете брать код из семинарского ноутбука, то поменяйте в нем что-нибудь, например, цветовую гамму, иначе проверяющий будет карать :)

Какой признак стоит исключить из рассмотрения корреляций? Напишите его название, а также причину, почему нет смысла его рассматривать.

Вам может пригодиться:
- sns.heatmap
- методы get_text и set_text у атрибута texts графика

In [ ]:
# Выводим таблицу корреляций с помощью heatmap, используя датасет train
plt.figure(figsize=(12, 8))
table = sns.heatmap(train.corr(numeric_only=True), cmap="crest", annot=True, fmt="0.2f")
for value in table.texts:
  if float(value.get_text()) >= 0.5 or float(value.get_text()) <= -0.5: value.set_text(value.get_text())
  else: value.set_text("")
plt.title("Correlation heatmap")
plt.show()

# Из рассмотрения стоит исключить признак subject, так как его значения никак не коррелируют с остальными признаками,
# ведь номер испытуемого никак не зависит от других признаков и не связан с ними

### Задание 8 (0.5 балла)

Для начала будем стрелять из пушки по воробьям. Выведите pairplot по всем данным train сета. Напишите, заметили ли вы в нем что-нибудь интересное, чего не было в таблицах и графиках ранее, а также сообщите, было ли вам удобно анализировать 100 графиков.

In [ ]:
# Построим pairplot по всем данным датасета train
sns.pairplot(train);

# Лишь на некоторых графиках можно увидеть, что определенные признаки хорошо коррелируют, однако анализировать 100 графиков очень неудобно

### Задание 9 (1 балл)

Окей. Смотреть на все сразу не очень удобно. Кажется, нужно смотреть на признаки в отдельности и искать в них инсайты.

Попробуем проверить гипотезу. Отобразите плотность распределения (kde) `tBodyAccMag-mean()` для каждой из активностей, как в примере ниже. Видите закономерность? Можно ли поделить все активности на две группы? Если да, то проведите вертикальную линию, которая визуально неплохо разделяет эти группы.

Давайте первую группу называть статическими активностями, а вторую - динамическими.

Посчитайте, какую долю правильных ответов дает ваша вертикальная линия для разделения именно этих двух групп. С чем может быть связан полученный вами результат? Предположите логическое обоснование (почему именно данный признак так хорошо/плохо разделяет группы активностей).

Вам может пригодиться:
- sns.kdeplot

In [ ]:
# Строим график плотности распределения усредненной величины ускорения для разных активностей датасета train с помощью kdeplot
attribute = 'tBodyAccMag-mean()'
plt.figure(figsize=(15, 5))
sns.kdeplot(train, hue='Activity', x =attribute, palette='dark')
plt.title('Mean body acceleration for activities')
plt.xlabel('Body acceleration')
plt.ylabel('Density')
plt.legend(title='Activities', labels=['WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'WALKING', 'LAYING', 'SITTING', 'STAYING'])
plt.axvline(x=-0.5)
plt.show()

# Можно увидеть закономерность, что все виды активностей поделились на 2 группы,
# слева от вертикальной прямой - статические, а справа от нее - динамические

In [ ]:
# Разделим активности на две группы, и для всех результатов подсчитаем корректность ответа
static_activities = ['STANDING', 'SITTING', 'LAYING']
dynamic_activities = ['WALKING', 'WALKING_DOWNSTAIRS', 'WALKING_UPSTAIRS']
static_true_results = ((train[attribute] <= -0.5) & (train['Activity'].isin(static_activities))).sum()
dynamic_true_results = ((train[attribute] > -0.5) & (train['Activity'].isin(dynamic_activities))).sum()
true_results = static_true_results + dynamic_true_results
results = train.shape[0]
ans = true_results / results
print('Доля правильных ответов:', ans)

# Доля правильных ответов оказалась очень высокой, полученный результат связан с тем, что данный признак очень сильно отличается
# в группе статических активностей и в группе динамических активностей. Это вызвано тем, что в статических активностях ускорение
# практически неизменно, в то время как у динамических активностей, оно меняется очень сильно и принимает другие значения

### Задание 10 (0.5 балла)

Поздравляем, вы проверили первую гипотезу в процессе EDA. Однако нам бы хотелось уметь не группы разделять, а каждый отдельный вид активности. Попробуйте нарисовать "ящики с усами" для `angle(X,gravityMean)` с разделением по разным видам активности на данных train. Можно ли сделать какой-то вывод по полученной визуализации? Проанализируйте и проинтерпретируйте полученные картинки (если заметили какую-то особенность, может какая-то категория значительно выделяется, то предположите, почему так могло произойти).

Вам может пригодиться:
- sns.boxplot

In [ ]:
# Строим "ящики с усами" для angle(X,gravityMean) на видах активностей датасета train с помощью boxplot
attribute = 'angle(X,gravityMean)'
plt.figure(figsize=(15, 5))
sns.boxplot(train, x='Activity', y=attribute, palette='magma')
plt.title('Angle between X-axis and gravitiational acceleration')
plt.xlabel('Activity')
plt.ylabel('Angel')
plt.show()

# Из полученной визуализации можно сделать вывод, что значения для всех активностей кроме LAYING лежат примерно в одном диапазоне, однако LAYING
# значительно выделяется на их фоне. Это может быть связано с тем, что именно, когда человек лежит,
# то  угол между осью X и усредненной гравитационной составляющей ускорения принимает наибольшее значение, ведь во время других активностей
# человек находится в другом положении относительно оси X, нежели во время лежания

### Задание 11 (0.5 балла)

Посмотрим теперь на `tBodyAcc-max()-X`. Визуализируйте скрипичные диаграммы для этого признака с разделением по активностям на всем train. Кажется логичным, что максимальное ускорение в моменте для разных активностей будет различаться. Однако есть ли активность, которая будет выделяться? Если да, то предложите объяснение, почему эта активность выделяется.

Вам может пригодиться:
- sns.violinplot

In [ ]:
# Строим график для tBodyAcc-max()-X с разделением по активностям датасета train, используя violinplot
attribute = 'tBodyAcc-max()-X'
plt.figure(figsize=(15, 5))
sns.violinplot(data=train, y=attribute, x ='Activity', palette='magma', split=True)
plt.title('Maximal body acceleration over activities')
plt.xlabel('Activity')
plt.ylabel('Maximal acceleration')
plt.show()

# Можно заметить, что выделяется такая активность, как WALKING_DOWNSTAIRS. Она принимает максимальные значения,
# это может быть связано с тем, что при спуске человек набирает дополнительное ускорение, тем самым достигая более высокого
# максимального значения, в отличие от статических активностей, где максимальное ускорение максимально и других динамических активностей,
# где максимальное ускорение немного меньше

### Задание 12 (1 балл)

Вы уже можете с довольно хорошим качеством выделить две категории активности из шести. Продолжим.

Ранее уже упоминалось разделение на две группы активностей: статические и динамические. Пришла пора разделить датасет train на две части. Сделайте это! Теперь на одной картинке покажите распределения `fBodyAcc-bandsEnergy()-1,8.2` для динамичных активностей (boxenplot - немного измененый "ящик с усами") и `angle(Y,gravityMean)` для статических - тут хватит и простой гистограммы (прямо, как в примере ниже). Можно ли выделить еще какие-то активности? Если да, напишите, какие активности на 1 и 2 графиках выделяются.

Вам может пригодиться:
- plt.subplots или matplotlib.gridspec (заметили, что один график больше другого? попробуйте сами так сделать - вам понравится...)
- sns.boxenplot
- sns.histplot

In [ ]:
# Разделяем датасет на две части в зависимости от группы активностей датасета train и рисуем график распределения
# fBodyAcc-bandsEnergy()-1,8.2 для динамичных активностей c помощью boxenplot и angle(Y,gravityMean) для статических с помощью histplot
train_static = train[train['Activity'].isin(static_activities)]
train_dynamic = train[train['Activity'].isin(dynamic_activities)]

attribute1, attribute2 = 'fBodyAcc-bandsEnergy()-1,8.2', 'angle(Y,gravityMean)'
fig, (first, second) = plt.subplots(1, 2, figsize=(15, 5), gridspec_kw={'width_ratios': {8, 5}})
fig.suptitle('Angel and Energy for activity groups')
sns.boxenplot(data=train_dynamic, ax=first, x='Activity', y=attribute1)
first.set_title('Dynamic activities')
first.set_ylabel('Frequency energy')
sns.histplot(data=train_static, ax=second, hue='Activity', x=attribute2, bins=70)
second.set_title('Static activities')
second.set_xlabel('Angel')
plt.show()

# На первом графике среди динамических активностей выделяется WALKING_UPSTAIRS, так как большинство ее даже минимальных значений
# превосходят значения двух других активностей. На втором графике, среди статических активностей выделяется STANDING, так как
# его значения сосредоточены лишь на небольшом промежутке

### Задание 13 (0.5 балла)

Не всегда жизнь будет такой легкой, а данные будут полностью или почти линейно разделимы в одному признаку (иначе чем бы занимались дата саентисты...). Часто данные вообще ни по одному признаку нельзя адекватно разделить (да и по комбинациям признаков тоже). Давайте попробуем взять не один признак, а целых два!

Рассмотрим только статические активности. Будет снова два графика сразу. На первом графике будет боксплот распределения `fBodyAcc-skewness()-X` по активностям. А на втором попробуем взять не все наблюдения данной группы активностей, а только их часть. Разделимы ли категории по `fBodyAcc-skewness()-X`? А если взять только те строки, где `fBodyAcc-skewness()-X < -0.376`, и теперь вывести боксплот распределения `angle(Y,gravityMean)` по разным активностям? Попробуйте и напишите, выделилась ли какая-то категория?

Вам может пригодиться:
- plt.subplots или matplotlib.gridspec
- sns.boxplot

In [ ]:
# Строим график распределения fBodyAcc-skewness()-X по статическим активностям датасета train с помощью boxplot
# и график распределения angle(Y,gravityMean) для строк, где fBodyAcc-skewness()-X < -0.376
# у статичесикх активностей с помощью boxplot
attribute1, attribute2 = 'fBodyAcc-skewness()-X', 'angle(Y,gravityMean)'
train_static_with_cond = train_static[train_static[attribute1] < -0.376]

fig, (first, second) = plt.subplots(1, 2, figsize=(15, 5))
fig.suptitle('Acceleration and Angle over static activities')
sns.boxplot(data=train_static, ax=first, x='Activity', y=attribute1, palette='magma')
first.set_title('Skewness of body acceleration over activities')
first.set_ylabel('Skewness')
sns.boxplot(data=train_static_with_cond, ax=second, x='Activity', y=attribute2, palette='dark')
second.set_title('Angle between Y-axis and gravity accleration over activities')
second.set_ylabel('Angle')
plt.show()

# На первом графике нельзя разделить активности друг друга, однако используя второй график это можно сделать достаточно легко.
# Можно выделить такую активность как SITTING, ее значения находятся в диапазоне, на который не попадают никаких другие активности

Поздравляем, теперь вы на шажок ближе к тому, чтобы стать решающим деревом!

### Задание 14 (1 балл)

Вот и осталась лишь одна активность, которую мы никак не выделили. Попробуем и ее какими-то порогами отделить от всех остальных. Да, можно было бы сказать, что, раз уж мы выделили 5 из 6 категорий, то все остальное - это и есть последняя категория, но мы не идем легкими путями. Выделить оставшуюсь категорию с хорошим качеством не так уж и просто. Из доступных признаков, сначала посмотрите на распределение `tBodyAcc-max()-X` на динамических активностях (скрипичная диаграмма отлично подойдет).

А теперь спускаем вам порог `tBodyAcc-max()-X < 0.157`. Для таких строк постройте диаграмму рассеяния для признака `tGravityAcc-min()-X`. Да, получается неидеально, но это честная работа. Можно выбрать порог так, чтобы чисто из таблицы с динамическими активностями из строк, где `tBodyAcc-max()-X < 0.157`, выбрать по некоторому порогу для `tGravityAcc-min()-X` строки так, чтобы доля правильных ответов на оставшейся категории была не меньше 83%!

Вам может пригодиться:
- plt.subplots или matplotlib.gridspec
- sns.violinplot
- sns.swarmplot (иногда лучше подходит, чем просто sns.scatterplot)

Обратите внимание, что подписи к осям можно убирать методами plt. Например, если у вас общее название для оси Y для обоих графиков, то можно не подписывать ось Y у второго графика.

In [ ]:
# Строим график распределения tBodyAcc-max()-X по динамическим активностям датасета train с помощью violinplot
# и график распределения tGravityAcc-min()-X для строк, где tBodyAcc-max()-X < 0.157
# у динамических активностей с помощью swarmplot
attribute1, attribute2 = 'tBodyAcc-max()-X', 'tGravityAcc-min()-X'
train_dynamic_with_cond = train_dynamic[train_dynamic[attribute1] < 0.157]

fig, (first, second) = plt.subplots(1, 2, figsize=(15, 5))
fig.suptitle('Body and Gravity acceleration over dynamic activities')
fig.supylabel('Acceleration')
sns.violinplot(data=train_dynamic, ax=first, x='Activity', y=attribute1, palette='bright')
first.set_title('Maximal body acceleration over activities')
first.set_ylabel("")
sns.swarmplot(data=train_dynamic_with_cond, ax=second, x='Activity', y=attribute2, palette='magma')
second.set_title('Minimal gravity acceleration over activities')
second.set_ylabel("")
fig.tight_layout()
plt.show()

# На данных графиках достаточно тяжело выделить какой-либо признак, однако можно заметить, что только WALKING
# на правом графике принимает значения выше 0.97 и почти все значения выше 0.95 тоже относятся к этой категории

### Задание 15 (0.5 балла)

Теперь попробуем более интерактивные графики. Установите библиотеку `plotly` (обычно `pip install plotly` или `conda install plotly`). Синтаксис очень похож на `seaborn`, но вот [документация для plotly.express](https://plotly.com/python-api-reference/plotly.express.html).

Сначала подготовим данные для визуализации. Будем считать t-SNE просто "черным ящиком", который принимает наши данные и проецирует их в двумерное или трехмерное пространство. Возможно, придется немного подождать.

In [ ]:
# Устанавливаем библиотеку plotly
!pip install plotly

In [ ]:
# Импортируем модуль express библиотеки plotly
import plotly.express as px

In [ ]:
%%time
from sklearn.manifold import TSNE
tsne = TSNE(random_state=666, n_components=2)
new = tsne.fit_transform(train.drop(['subject', 'Activity'], axis=1))
df_2_comp = pd.concat([
    pd.DataFrame(new, columns=['x', 'y']),
    train['Activity']
], axis=1)

tsne = TSNE(random_state=666, n_components=3)
new = tsne.fit_transform(train.drop(['subject', 'Activity'], axis=1))
df_3_comp = pd.concat([
    pd.DataFrame(new, columns=['x', 'y', 'z']),
    train['Activity']
], axis=1)

Теперь нарисуйте scatter-график через `plotly.express` для `df_2_comp` с выделением цветами активностей. Красиво получилось? Видны скопления точек одного цвета? Можно разные активности отделить линиями на данном графике?

In [ ]:
# Строим scatter-график для df_2_comp с выделением цветами активностей
px.scatter(df_2_comp, x='x', y='y', color='Activity')

# Получилось достаточно красиво, ведь хорошо видны скопления точек одного цвета. Некоторые группы легко можно отделить линиями на графике,
# например, LAYING находится в отдалении от всех остальных, SITTING и STANDING можно постараться разделить линией, однако получится не идеально

Мы не отстаем от трендов и тоже можем в 3D. Сейчас и вы научитесь (если еще не умеете). Нарисуйте scatter_3d-график через `plotly.express` для `df_3_comp` с выделением цветами активностей. Повращайте его. Нормально получилось? Есть ли какая-то структура у точек?

In [ ]:
# Строим scatter-график для df_2_comp с выделением цветами активностей
px.scatter_3d(df_3_comp, x='x', y='y', z='z', color='Activity')

# Получилось красиво, несмотря на то, что точек много и они находятся близко друг к другу. Точки разных категорий находятся близко друг к другу
# и для каждой категории можно нарисовать окружность, покрывающую всю активность

### Задание 16 (1 балл)

Мы сделали вам fit. Ваша очередь сделать predict. Допишите функцию, которая принимает на вход датафрейм с признаками, а выдает вектор из предсказаний - для каждой строки датафрейма вы определяете, к какому классу принадлежит объект. Рекомендуем сразу записывать из строками, а не кодировать в числа. Дальше фукнция `accuracy_score` выдаст вам долю правильных ответов.

Подсказка: вы уже отделили категории друг от друга по каким-то признакам. Воспользуйтесь этим знанием! Используя только проведенный вами анализ можно спокойно выбить не менее 75% на train и test! Если вы получите меньше 75%, то балл будет снижен. Авторское решение (на основе всех графиков выше) выбивает 81% и 77% на train и test соответственно. [Шапргалка с готовыми идеями для такой задачи](https://www.youtube.com/watch?v=dQw4w9WgXcQ)

In [ ]:
# Заводим массив с предсказаниями и в зависимости от значений различных признаков, выбираем категорию активности, которую помещаем в массив ответов
def predict(df: pd.DataFrame):
    ans = []
    for row_number, row in df.iterrows():

      # статические активности
      if(row['tBodyAccMag-mean()'] < -0.5):
        if(row['angle(X,gravityMean)'] > 0.1):
          ans.append('LAYING')
        elif(row['angle(Y,gravityMean)'] > -0.35 and row['angle(Y,gravityMean)'] < 0.1):
          ans.append('SITTING')
        elif(row['angle(Y,gravityMean)'] > 0.13):
          ans.append('STANDING')
        else:
          ans.append('SITTING')

      # динамические активности
      else:
        if(row['tGravityAcc-min()-X'] < 0.8 or row['fBodyAcc-bandsEnergy()-1,8.2'] > 0.3):
          ans.append('WALKING_UPSTAIRS')
        elif(row['tBodyAcc-max()-X'] > 0.22):
          ans.append('WALKING_DOWNSTAIRS')
        elif(row['fBodyAcc-skewness()-X'] < -0.1 or row['tBodyAcc-max()-X'] < -0.41):
          ans.append('WALKING')
        else:
          ans.append('WALKING_UPSTAIRS')

    return ans

def accuracy_score(y_true, y_pred) -> float:
    y_true = np.asarray(y_true).reshape(-1, 1)
    y_pred = np.asarray(y_pred).reshape(-1, 1)
    return (y_true == y_pred).mean()

In [ ]:
train_preds = predict(train)
test_preds = predict(test)
print(f"Accuracy on train set => {accuracy_score(train['Activity'], train_preds)}")
print(f"Accuracy on test set => {accuracy_score(test['Activity'], test_preds)}")

# Итоговые значения на train = 81% и на test = 77% что больше 75%

__Место для мема и/или фидбэка__